# Шаблоны классов

## Проблема “одинаковых классов”

    struct  ArrayInt {
        explicit  ArrayInt(size_t  size)        // констр. от размера
        : data_(new  int[size])                 // дин.массив от размера
        , size_(size) 
        {}
        ~ArrayInt () { delete  [] data_; }      // дестр.
        size_t  size() const { 
            return  size_; 
        }
        int  operator []( size_t i) const {     // конст.опер.[]
            return  data_[i]; 
        }
        int & operator []( size_t i) {          // не конст.опер []
            return  data_[i]; 
        }
        ...
    private:
        int *    data_;                         // данные
        size_t   size_;
    };

    // для float практически все совпадает, кроме имени и типа 
    struct  ArrayFlt {
        explicit  ArrayFlt(size_t  size)
        : data_(new  float[size]), size_(size) 
        {}
        ~ArrayFlt () { delete  [] data_;}
        size_t  size() const { 
            return  size_; 
        }
        float  operator []( size_t i) const { 
            return  data_[i]; 
        }
        float & operator []( size_t i) { 
            return  data_[i]; 
        }
        ...
    private:
        float   * data_;
        size_t    size_;
    };



## Решение в стиле C: макросы


В С нет шаблонов (как и классов).

    #define  DEFINE_ARRAY(Name , Type)  \   // макрос с параметрами (Name , Type)
    struct  Name {                      \   // которые заменятся в тексте на аргументы
    explicit  Name(size_t  size)        \
    : data_(new  Type[size])            \   // макрос по определению однострочный, поэтому тут через \
    , size_(size) {}                    \
    ~Name() { delete  []  data_; }      \
                                        \
    size_t  size() const                \
    { return  size_; }                  \
                                        \
    Type  operator []( size_t i) const  \
    { return  data_[i]; }               \
    Type & operator []( size_t i)       \
    { return  data_[i]; }               \
    ...                                 \
    private:                            \
    Type *   data_;                     \
    size_t   size_;                     \
    }


    DEFINE_ARRAY(ArrayInt , int);           // препроцессор тут вставит текст по макросу
    DEFINE_ARRAY(ArrayFlt , float);
    int  main()
    {
        ArrayInt  ai(10);
        ArrayFlt  af(20);
        ...
        return  0;
    }

Минус тут главный один - это подстановки чисто на уровне текста. Никакие типы, имена и т.д. не проверяются.

## Решение в стиле C++: шаблоны классов

Преимущество в том, что обрабатываются на этапе компиляции (т.е. являются конструкцией языка, а не текстовым "инструментом"). 

    template  <class  Type>                     // < параметры шаблона > - class означает тип (эквивалентно typename)
    struct  Array {
        explicit  Array(size_t  size)
        : data_(new  Type[size]), size_(size) 
        {}
        ~Array() { delete  []  data_; }
        size_t  size() const { return  size_; }
        Type  operator [](size_t i) const {     // при компиляции будет проверено, что Type это именно тип
            return  data_[i]; 
        }
        Type & operator [](size_t i) {
            return  data_[i]; 
        }
        ...
    private:
        Type *   data_;
        size_t   size_;
    };


    int  main()
    {                                           
        Array <int> ai(10);                     // после компиляции это будет массив int
        Array <float> af(20);                   // ... float
        ...                                     // после первой компиляции шаблона, он может использоваться дальше
        
        return  0;
    }

NB: Шаблон класса нельзя перегузить, чтобы выбирался шаблон под конкретный набор шаблонных параметров.
    

## Шаблоны классов с несколькими параметрами

    template  <class  Type,                     // класс дин.массива типа Type
               class  SizeT = size_t,           // тип хранения размера (с умолчанием) 
               class  CRet = Type>              // тип возвращаемого значения (с умолчанием)
    struct  Array {
        explicit  Array(SizeT  size)
        : data_(new  Type[size]), size_(size) 
        {}
        ~Array() { delete  []  data_; }
        SizeT  size() const {return  size_ ;}
        CRet  operator []( SizeT i) const {     // для больших объектов возврат имеет смысл заменить на конст. ссылку
            return  data_[i]; 
        }
        Type & operator [](SizeT i) {
            return  data_[i]; 
        }
        ...
    private:
        Type *   data_;
        SizeT    size_;
    };

    void  foo() 
    {
        Array <int > ai(10);
        Array <float > af(20);
        Array <Array <int >,                    // массив массивов
               size_t, 
               Array <int> const & >            // возвр.конс.ссылку на массив, чтоб не копировать
            da(30);
        ...
    }
    
    typedef  Array <int> Ints;                  // синонимы типа (для улучшения читабельности)
    typedef  Array <Ints, size_t, 
                    Ints const &> 
                IInts;
    
    void  bar()
    {
        IInts  da(30);                          // по шаблону класса через синоним типа 
    }

*) сейчас в стандарте c++17 вместо typedef используется using

# Шаблоны функций

## Шаблоны функций: возведение в квадрат

    // C (тут просто разные имена)
    int    squarei(int   x)    { return x * x; }
    float  squaref(float x)    { return x * x; }
    
    // C++ (перегрузка по типу аргумента)
    int    square(int   x)   { return x * x; }                // +/- (простота)/(дублирование кода)
    float  square(float x)   { return x * x; }
    
    // C++ + OOP (переопределение - виртуальные методы)
    struct  INumber {
        virtual   INumber * multiply(INumber * x) const = 0;
    };
    struct  Int : INumber { ... };                            // int/float классами не являются, поэтому нужно написать класс-обертку
    struct  Float : INumber { ... };
    
    INumber * square(INumber * x) { return x->multiply(x); }  // +/- (функция всего одна) / (муторные обертки, полиморфизм снижает производительность)
    
    // C++ + templates 
    template  <typename Num >Num  square(Num x) { return x * x; }

## Шаблоны функций: сортировка

    // C (встроенная функция, работающая через void*, размер элемента и указатель на function сравнения)
    void  qsort(void * base , size_t  nitems , size_t  size , /* function */);  // о как, есть оказывается указатели на функции
                                                                                // что интересно, обмен элементов происходит просто побайтным обменом
                                                                                // => для объектов побайтовое копирование считается UB, поэтому не подходит для них 
    
    // C++
    void  sort(int    * p, int    * q);
    void  sort(double * p, double * q);
    
    // C++ + OOP (аналогично через интерфейс)
    struct  IComparable {
        virtual   int  compare(IComparable * comp) const = 0;
        virtual ~IComparable () {}
    };

    void  sort(IComparable ** p, IComparable ** q); // указатель на массив указателей (в *p указатель на значение)
    // подход плох тем, что со всеми объектами нужно работать полиморфно через указатели, а это большие накладные расходы
    // плюс для встроенных типов потребуются классы-обертки
    
    // C++ + templates
    template <typename Type> void sort(Type * p, Type * q); // будет работать со всем типами, где есть оп. <
    // второй указатель - на конец массива, чтобы было удобнее делать циклы
    
NB: у шаблонных функций нет параметров по умолчанию.
Т.к. для них не запрещена перегрузка (шаблоны классов не перегружаются), то можно уменьшить число параметров за счет перегрузки: в одном варианте - одна часть параметров захардкодена, в другом - другая, и можно вызывать их друг из друга.

## Вывод аргументов (deduce) 

Если по шаблонным параметрам можно однозначно определить подходящий к вызову шаблон, то это будет сделано компилятором.

    template  <typename Num >
    Num  square(Num n) { return n * n; }
    
    template  <typename  Type >
    void  sort(Type * p, Type * q);
    
    template  <typename  Type >             // перегрузка шаблонной функции sort для Array
    void  sort(Array <Type > & ar);         // шабл.пар.Type не участвует явно в типах аргументов (ф-я принимает тип, производный от Type)
    
    void  foo() {
        int a = square <int >(3);           // явно указали
        int b = square(a) + square (4);     // deduction -> square<int>(..) (явно указывать не надо)
        float * m = new  float [10];
        sort(m, m + 10);                    // deduction -> аргумент float* -> sort<float>(m, m + 10)
        sort(m, &a);                        // error: sort<float> vs. sort<int> (уравнение на типы не решается Type!=Type)
        Array <double > ad (100);
        sort(ad);                           // deduction -> аргумент Array<double> -> sort<double>(ad)}

Компилятор может выводить тип, даже если аргумент не является непосредственно (подставляемым) типом, но и если он является однозначно определяемым производным типом от шаблонного параметра. Просто конструирует тип функции из шаблонного параметра, вроде тривиально, в чем прикол?

Среди перегружаемых операторов в C++ есть оператор (). Рассмотрим пример перегрузки:

    struct Less { 
        bool operator()(int a, int b) const { return a < b; } 
    };

Создадим объект Less и назовем его less. Использование этого объекта синтаксически выглядит как вызов функции:

    Less less;
    
    if (less(10, 20)) 
        std::cout << "10 < 20" << std::endl;
    else 
        std::cout << "10 >= 20" << std::endl;

Если заменить объект less на функцию с тем же именем:

    bool less(int a, int b) { return a < b; }

то код выше не потребует никаких изменений. Однако, хотя в обоих случаях объекты (на самом деле объект и функция) имеют одно и то же имя (less), их типы отличаются. 

*) первая будет Less::less(10,20), вторая ::less(10,20)

Шаблоны помогают преодолеть различие между объектами и функциями. Можно добавить в функции sort дополнительный параметр, который задаёт отношение меньше:

    template<class T, class Comp>
    void sort(T * p, T * q, Comp less);


В качестве третьего параметра в эту функцию Вы можете передать как функцию (на самом деле при этом будет передан указатель на функцию), так и объект класса с перегруженным оператором  "()". И при этом вам не нужно знать его конкретный тип, важно лишь то, что его использование синтаксически похоже на вызов функции.

    int m[10] = {0,1,2,3,4,5,6,7,8,9};

    sort(m, m + 10, Less()); // передаём объект типа Less с перегруженными ()
    sort(m, m + 10, less);   // передаём функцию less

    // а еще с С++11 стало можно так (что-то типа лямбды)
    sort(m, m + 10, [](int x, int y){return x < y;});

## Шаблоны методов

Дополним шаблонный класс массива шаблонным "конструктором копирования" и "шаблонным оператором присваивания".  

Почему в "". Системообразующие методы класса (конструктор по умолчанию, конструктор копирования, оператор присваивания и деструктор), а также любые виртуальные методы - НЕ МОГУТ БЫТЬ ШАБЛОННЫМИ. Запрещено стандартом.  

Это нужно для инициализации копированием/присваиванием другого класса шаблонного массива, если существует соответствующее приведение типа, например:

    Array<int> a;
    Array<double> b(a);
    Array<double> c = a;
    a = b;  // и обратно также ок, потому что у него тоже все эти шаблонные методы есть

Для этого и добавляются шаблонные конструкторы (которые не конструкторы копирования/присваивания, потому что тип другой):

    template  <class  Type >
    struct  Array {
        template <class  Other >                    // даже если тут указать Type, придется все равно делать настоящие конструкторы
        Array( Array <Other > const& other )        // а так при вызове К.копирования получится несовпадение типов и задействуется шаблон
        : data_(new  Type[other.size ()])
        , size_(other.size ()) {
            for(size_t i = 0; i != size_; ++i)
                data_[i] = other[i];
        }
        template <class  Other>
        Array & operator =(Array <Other > const& other );   // аналогично, плюс еще доступно обратное преобразование 
        ...
    };
    
    // как и обычные методы, шаблонные тоже можно определять за пределом класса
    // только в случае шаблонного метода шаблонного класса будет чуть более заковыристый синтаксис:
    
    template <class  Type>      // двойной шаблонный заголовок (шаб.параметры шаблонного класса)
    template <class  Other>     // (шаб.параметры шаблонного метода)
    Array <Type > & Array <Type >::operator =(Array <Other > const& other)      // при реализации указ. полное название шабл.типа
    { ...  return *this; }

# Специализация шаблонов

## Полная специализация шаблонов: классы

Когда шаблон без параметров и полностью прописана его реализация под специфическую задачу.  

Адаптируем шаблонный класс массива к работе с бул.типом с учетом выравнивания.  
Давайте хранить булы в интах. Все просто, элементы - инты, индекс по битам.

    template <class T>
    struct  Array {
        ...
        T *    data_;                   // для хранения массива нужно sizeof(T) * size байт
    };
    
    template <>                                                     // шаблон без параметров - все параметры в теле шаблона
    struct  Array <bool > {
        static  int  const  INTBITS = 8 * sizeof(int);              // сколько бит в 1 инте
        explicit  Array(size_t  size)
            : size_(size)
            , data_(new  int[size_ / INTBITS + 1])                  // +1 т.к. инты и целочисленное деление
        {}
        bool  operator []( size_t i) const {                        // константный - т.е. можем вернуть по значению (а ссылку на бит для неконстантного не получится (простым образом))
            return  data_[i / INTBITS] & (1 << (i % INTBITS ));     // битовая арифметика - данные по индексу инта с нужным битом, по маске И, сдвинутой на остаток, вроде понятно)
                                                                    // аналогично выставляются биты - все ясно (маска из нулей с 1  - истина, наоборот их единиц с 0 - ложь)
        }
    private:
        size_t   size_;
        int *   data_;
    };

*) по памяти мы выиграли в минимум в 8 раз, по доступу проиграли, но не сильно, изза раскодирования интов в булы

## Полная специализация шаблонов: функции

Аналогично для функций - шаблон без параметров или кастомный вид шаблона под задачу. 

    template <class T>                                      // обычный шаблон для функции
    void  swap(T & a, T & b)
    {
        T tmp(a);
        a = b;
        b = tmp;
    }
    
    template <>                                             // специальный вариант swap где нет К.копирования и оп.=
    void swap <Database >( Database & a, Database & b)
    {
        a.swap(b);
    }
    
    template <class T>                                      // перегрузка для массивов
    void  swap(Array <T> & a, Array <T> & b)                // например у элементов есть К.копир и оп.= но такой подход заставит копироваться каждый элемент
    {
        a.swap(b);                                          // а нам нужно просто поменять указатели
    }

## Специализация шаблонов функций и перегрузка

В чем же разница между полной специализацией шаблонных функций и перегрузкой шаблонных функциий.  
Определим три шабл. функции

    template <class T>                                      // определили
    void  foo(T a, T b) {
        cout  << "same  types" << endl;
    }
    
    template <class T, class V>                             // перегрузили
    void  foo(T a, V b) {
        cout  << "different  types" << endl;
    }
    
    template <>                                             // полная специализация под аргументы int int
    void foo <int , int >(int a, int b) {
        cout  << "both  parameters  are  int" << endl;
    }
    
    // фишка в том, что сначала полностью отрабатывается перегрузка
    int  main() {
        foo(3, 4);  // будет выбираться из 2 перегружаемых вариантов - подходит первый вариант, он и будет использован

        foo <int,int> (3,5); // третий вариант можно вызвать только явно передав шаблонные параметры

        return  0;
    }

Поэтому для функций использование шаблонов полной специализации - довольно спорный момент. Проще написать эту конкретную специализацию в виде нешаблонной функции, она тоже будет участвовать в перегрузке и будет выбрана как наиболее подходящая по сигнатуре:  

- приоритет всегда отдается нешаблонным функциям. Если находится нешаблонная функция, которая удовлетворяет всем параметрам, то вызовется именно она;
- если среди нешаблонных функций не нашлось подходящей, то компилятор рассматривает различные перегрузки базового шаблона (base template function), не учитывая специализаций, и выбирает среди перегрузок;
- если какая-то из перегрузок базового шаблона подошла, то далее компилятор проходится по ее специализациям и выбирает среди них.

## Частичная специализация шаблонов


У классов не бывает перегрузок (не методов а классов, класс только один). Перегружаются именно шаблоны классов.  


    template <class T>                                          // основной шаблон массива
    struct  Array {
        T & operator []( size_t i) { return  data_[i]; }
        ...
    };
    
    template <class T>                                          // специализируем его для работы с массивами указателей
    struct  Array <T *> {
        explicit  Array(size_t  size)
            : size_(size)
            , data_(new T *[size_ ])
        {}
        
        T & operator []( size_t i) { return *data_[i]; }        // отличие в возврате разыменованного указателя
    private:
        size_t   size_;
        T **     data_;
    };

Вопрос с дублированием кода при таком подходе может решаться за счет наследования частично специализированного шаблонного класса от базового.  

Специализация шаблонов для получения информации о типах используется в C++ довольно активно. Например, для простых типов вроде char или int функция copy_n (которую вы реализовали раньше) может использовать memcpy. Использование memcpy может оказаться быстрее, чем поэлементное копирование массива.

Давайте создадим шаблон TriviallyCopyable, который позволит узнать, допустимо ли для данного типа побайтовое копирование вместо вызова конструктора копирования или оператора присваивания. Как может выглядеть такой шаблон:

    template <typename T>
    struct TriviallyCopyable
    { static const bool value = false; };

Если значение value == true, значит для копирования можно использовать memcpy. В шаблоне выше значение value по умолчанию равно false, потому что безопасно считать, что тип нельзя копировать с использованием memcpy (возможно, мы немного проиграем от этого в скорости, но не в правильности).

Функция copy_n может теперь использовать значение TriviallyCopyable<T>::value, чтобы выбрать общую реализацию или реализацию с использованием memcpy. Однако, чтобы от TriviallyCopyable была польза, нужно специализировать его так, чтобы для простых типов value было равно true, например, так:

    template <> struct
    TriviallyCopyable<char>
    { static const bool value = true; };

Аналогичным образом, мы можем специализировать TriviallyCopyable для типов (классов), определенных пользователем (если их, конечно, действительно можно копировать с использованием memcpy). Выигрыш по сравнению со специализацией/перегрузкой функции copy_n заключается в том, что класс TriviallyCopyable может использовать не только copy_n, а любая другая функция или класс, для которой эта информация может оказаться полезной.


*) static поля класса можно инициализировать только снаружи классов, кроме случа static const


# Ещё о шаблонах

## Нетиповые шаблонные параметры

Параметрами шаблона могут быть типы, целочисленные значения, указатели/ссылки на значения с внешней линковкой и шаблоны.

    template <class T, size_t N, size_t M>                      // целочисленные шабл.пар. (опр.на этапе компиляции)
    struct  Matrix {
        ...
        T & operator ()( size_t i, size_t j)
        { return  data_[M * j + i]; }
    private:
        T data_[N * M];
    };
    
    template <class T, size_t N, size_t M, size_t K>
    Matrix <T, N, K> operator *(Matrix <T, N, M> const& a,      // в оп.* попадут только матрицы с сопряженной размерностью N*M, M*K
                                Matrix <T, M, K> const& b);     // иначе функция не вызовется в принципе
    
    // log - это глобальная переменная
    template <ofstream & log >                                  // указ./ссылки в параметрах (опр. на этапе внешней линковки)
                                                                // для этого они должны быть глобальными перем. или стат. полем класса
    struct  FileLogger { 
        FileLogger() { log << "Logging started.\n"; }
        ostream &PutRecord(string record) { return log << record; }
       ~FileLogger() { log << "Logging finished.\n"; }
    };

    int main() {
        FileLogger<cout> fl1;                                   // можно создать разные с разными файлами/потоками
        fl1.PutRecord("Logging to fl1\n");
        return 0;
    }

## Шаблонные параметры — шаблоны

    // int –> string
    string  toString( int i );                              // пусть есть функция
    
    // работает только с Array<>
    Array <string > toStrings( Array <int > const& ar ) {   // определим для конкретного шаблонного класса массива ее специальный вариант (перегрузкой)
        Array <string > result(ar.size ());                 // конструктор этого массива по параметру size
        for (size_t i = 0; i != ar.size (); ++i)
            result.get(i) = toString(ar.get(i));            // заполним
        return  result;                                     // вернем
    }                                                       // не универсально в общем, только для 1 варианта шаблона
    
    // сделаем более универсальный вариант (для почти любого контрейнера)
    // от контейнера требуются: конструктор от size, методы size() и get()

    template <template  <class > class  Container >         // шаблонный пар. шаблона задает как раз требования к контейнеру, чтоб 
                                                            //..компилятор при выводе аргументов наткнулся именно на этот шаблон
    Container <string > toStrings( Container <int > const& c) { // эт. че, получается компилятор при выводе аргументов смотрит, есть
                                                                // ли у "c" вызываемые в функции методы с.size() c.get() ?
        Container <string > result(c.size ());                  // ..какой молодец...
        for (size_t i = 0; i != c.size (); ++i)
            result.get(i) = toString(c.get(i));
        return  result;
    }

## Использование зависимых имён

Еще одна техника:

    template <class T>
    struct Array {
        typedef T value_type;                   // определим синоним для T, тогда в коде Array<int>::value_type -> int
        ...
    private:
        size_t   size_;
        T *      data_;
    };
    
    template <class  Container >
    bool  contains(Container  const& c,         // тогда по имени можем невозбранно получать имя хранящегося типа
                   typename  Container::value_type  const& v);
                   
    int  main()
    {
        Array <int > a(10);
        contains(a, 5);
        return  0;
    }

Есть такая проблемка у зависимых имен, т.к. синтаксис в целом проверяется раньше компиляции шаблона, если не указать typename и не дать тем самым понять компилятору, что параметр это тип, он может принять его просто за стат. поле класса (а не синоним/алиас typedef), и чето не то подставить или пропустить этот шаблон. Поэтому для таких зависимых имен (т.е. имена параметров зависят от шаблонного параметра) typename обязательно.

*) путаница в том, что синтаксис  Container::value_type это обращение к полю класса, а такого поля там нет, на этапе проверки синтакиса все завалится без typename

## Использование функций для вывода параметров

Для функций позволяется не использовать шабл.пар. если они могут быть однозначно выведены при вызове функции. Для шаблонных классов такой механизм не работает (не работал до С++17), но можно использовать функцию для такого автоматического вывода параметров для классов. 

    template <class  First , class  Second >                        // шаблонный класс
    struct  Pair {                                                  // спаривает два значения в одной структуре
            Pair(First  const& first , Second  const& second)
            : first(first), second(second)                          // конструктор инициализирует поля
            {}
            
            First  first;
            Second  second;
    };
    
    template <class  First , class  Second >                        // шабл.функция для автовывода параметров класса
    Pair <First , Second > makePair(First  const& f, Second  const& s) {
        return  Pair <First , Second >(f, s);                       // + такая функция как правильно довольно простая, чтобы        
                                                                    // компилятор мог догадаться заинлайнить ее, а не вызывать
    }
    
    void  foo(Pair <int , double > const& p);                       // если сделать нешаблонную ф-ю..
    
    void  bar() {
        foo(Pair <int , double >(3,  4.5));                         // .. то для класса нужно вызывать дублируя типы 
                                                                    // (объясняется тем, что у класса может быть несколько шаблонных конструкторов)
        foo(makePair(3, 4.5));                                      // а вот так через шаблонную функцию (по факту makePair<int,double>) все красиво

        foo(Pair(3, 4.5));  // C++17 сделали class template argument deduction, и можно так 
    }

## Компиляция шаблонов

∙Шаблон независимо компилируется для каждого значения шаблонных параметров.  

∙Компиляция (инстанциирование) шаблона происходит в точке первого использования — точке инстанциирования шаблона.  

    Array<double> a(10);    // первое упоминание шаблона в коде - точка инстанциирования
    Array<double> b;        // тут и далее этот вид шаблона уже инстанциирован

∙Компиляция шаблонов классов — ленивая, компилируютсятолько те методы, которые используются.  

    Array<double> a(10);    // причет тут будет скомпилирован только подходящий конструктор от параметра и деструктор
                            // поэтому могут быть непроявленные ошибки, если какой-то метод шаблона (по синтаксису нормальный) 
                            // не умеет работать с double
                            // об этом на этапе компиляции никто не узнает

∙В точке инстанциирования шаблон должен быть полностью определён.  

    Поэтому нельзя разделить объявление шаблона в заголовках, определение в коде (!), и поэтому:

∙Шаблоны следует определять в заголовочных файлах.  

∙Все шаблонные функции (свободные функции и методы) являются inline.  

∙В разных единицах трансляции инстанциирование происходит независимо.  

    В каждой единице трансляции у каждого шаблона будет своя точка инстанциирования.
    Статические поля шаблонных классов обрабатываются подобно inline-методам:
        - сначала каждая единица трансляции имеет свою версию static-поля,
        - затем компоновщик оставляет единственную?

## Резюме про шаблоны

∙Большие шаблонные классы следует разделять на два заголовочных файла: объявление (array.hpp) и определение(array_impl.hpp).   

    В программе в начале идет #include "array.hpp"
    В конце array.hpp добавляется #include "array_impl.hpp"

∙Частичная специализация и шаблонные параметры по умолчанию есть только у шаблонов классов.  

∙Вывод шаблонных параметров есть только у шаблонов функций.  

∙Предпочтительно использовать перегрузку шаблонных функций вместо их полной специализации.  

∙Полная специализация функций — это обычные функции, полная специализации класса - обычный класс.  

    Если такие функции определены в заголовочном файле, то надо их либо делать inline, либо разделять на обычное объявление/определение.
    

∙Виртуальные методы, конструктор по умолчанию, конструктор копирования, оператор присваивания и деструктор не могут быть шаблонными. 

    Шаблоны и ООП не дружат.
    
∙Используйте typedef для длинных шаблонных имён.  